In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/models1/pp_model.h5
/kaggle/input/efficient-net/archive/PyWavelets-1.1.1-cp37-cp37m-manylinux1_x86_64.whl
/kaggle/input/efficient-net/archive/h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl
/kaggle/input/efficient-net/archive/matplotlib-3.3.2-cp37-cp37m-manylinux1_x86_64.whl
/kaggle/input/efficient-net/archive/efficientnet-1.1.1-py3-none-any.whl
/kaggle/input/efficient-net/archive/six-1.15.0-py2.py3-none-any.whl
/kaggle/input/efficient-net/archive/networkx-2.5-py3-none-any.whl
/kaggle/input/efficient-net/archive/python_dateutil-2.8.1-py2.py3-none-any.whl
/kaggle/input/efficient-net/archive/tifffile-2020.9.22-py3-none-any.whl
/kaggle/input/efficient-net/archive/numpy-1.19.2-cp37-cp37m-manylinux2010_x86_64.whl
/kaggle/input/efficient-net/archive/pyparsing-2.4.7-py2.py3-none-any.whl
/kaggle/input/efficient-net/archive/cycler-0.10.0-py2.py3-none-any.whl
/kaggle/input/efficient-net/archive/certifi-2020.6.20-py2.py3-none-any.whl
/kaggle/input/efficient-net/archive/kiwisolver-1.2.0

In [2]:
!pip install ../input/efficient-net/archive/Keras_Applications-1.0.8-py3-none-any.whl
!pip install ../input/efficient-net/archive/efficientnet-1.1.1-py3-none-any.whl

Processing /kaggle/input/efficient-net/archive/Keras_Applications-1.0.8-py3-none-any.whl
Processing /kaggle/input/efficient-net/archive/efficientnet-1.1.1-py3-none-any.whl


In [3]:
import tensorflow as tf
from tensorflow.keras import layers
import efficientnet.tfkeras as efn
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import PIL
import shutil
import csv
from tensorflow.keras.applications.efficientnet import preprocess_input

In [4]:
BASE_DIR = '../input/plant-pathology-2021-fgvc8'
TRAIN_PATH = "../input/plant-pathology-2021-fgvc8/train_images/"
train_df = pd.read_csv(os.path.join(BASE_DIR, 'train.csv'), nrows=5000)
validate_df = pd.read_csv(os.path.join(BASE_DIR, 'train.csv'), skiprows=range(1,5000), nrows=5000)

#test_df = pd.read_csv(os.path.join(BASE_DIR, 'train.csv'), skiprows=range(1,10000), nrows=1000)
test_df = pd.read_csv(os.path.join(BASE_DIR, 'sample_submission.csv'))

count_dict = train_df.labels.value_counts()
classes = list(count_dict.index)
classes_count = list(count_dict.values)
print("Number of unique labels: ",len(classes))

print(len(test_df))
test_df.head()


Number of unique labels:  12
3


,image,labels
0,85f8cb619c66b863.jpg,healthy
1,ad8770db05586b59.jpg,healthy
2,c7b03e718489f3ca.jpg,healthy


In [5]:
labels_list = list(set(train_df.labels))
labels_list.sort()
mapping = {label:i for i, label in enumerate(labels_list)}
print(labels_list, '\n',mapping)

train_df['labels_id'] = train_df['labels'].map(mapping)
print(train_df)

['complex', 'frog_eye_leaf_spot', 'frog_eye_leaf_spot complex', 'healthy', 'powdery_mildew', 'powdery_mildew complex', 'rust', 'rust complex', 'rust frog_eye_leaf_spot', 'scab', 'scab frog_eye_leaf_spot', 'scab frog_eye_leaf_spot complex'] 
 {'complex': 0, 'frog_eye_leaf_spot': 1, 'frog_eye_leaf_spot complex': 2, 'healthy': 3, 'powdery_mildew': 4, 'powdery_mildew complex': 5, 'rust': 6, 'rust complex': 7, 'rust frog_eye_leaf_spot': 8, 'scab': 9, 'scab frog_eye_leaf_spot': 10, 'scab frog_eye_leaf_spot complex': 11}
                     image                           labels  labels_id
0     800113bb65efe69e.jpg                          healthy          3
1     8002cb321f8bfcdf.jpg  scab frog_eye_leaf_spot complex         11
2     80070f7fb5e2ccaa.jpg                             scab          9
3     80077517781fb94f.jpg                             scab          9
4     800cbf0ff87721f8.jpg                          complex          0
...                    ...                            

In [6]:
num_labels = []
for label in labels_list:
    num_labels.append(train_df[train_df['labels']==label].count().labels)
print(num_labels) 
print(labels_list)
for i, label in enumerate(labels_list):
    print(f'{mapping[label]} {label} : {num_labels[i]}')

[431, 802, 34, 1166, 394, 27, 515, 21, 30, 1363, 171, 46]
['complex', 'frog_eye_leaf_spot', 'frog_eye_leaf_spot complex', 'healthy', 'powdery_mildew', 'powdery_mildew complex', 'rust', 'rust complex', 'rust frog_eye_leaf_spot', 'scab', 'scab frog_eye_leaf_spot', 'scab frog_eye_leaf_spot complex']
0 complex : 431
1 frog_eye_leaf_spot : 802
2 frog_eye_leaf_spot complex : 34
3 healthy : 1166
4 powdery_mildew : 394
5 powdery_mildew complex : 27
6 rust : 515
7 rust complex : 21
8 rust frog_eye_leaf_spot : 30
9 scab : 1363
10 scab frog_eye_leaf_spot : 171
11 scab frog_eye_leaf_spot complex : 46


In [7]:
TEST_FOLDER = '../input/plant-pathology-2021-fgvc8/test_images/'
images = os.listdir(TEST_FOLDER)
sub = pd.DataFrame(images, columns=['image'])
print(sub)

                  image
0  ad8770db05586b59.jpg
1  c7b03e718489f3ca.jpg
2  85f8cb619c66b863.jpg


In [8]:
NUM_CLASSES =  len(classes)
HEIGHT,WIDTH = 128,128
BATCH_SIZE = 32
SEED = 143
SPLIT = 0.2

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255.,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split = SPLIT,
    zoom_range = 0.2,
    shear_range = 0.2,
    vertical_flip = False
)

train_ds = datagen.flow_from_dataframe(
    train_df,
    directory = TRAIN_PATH,
    x_col = "image",
    y_col = "labels",
    target_size = (HEIGHT,WIDTH),
    class_mode='categorical',
    batch_size = BATCH_SIZE,
    subset = "training",
    shuffle = True,
    seed = SEED,
    validate_filenames = False
)

val_ds = datagen.flow_from_dataframe(
    validate_df,
    directory = TRAIN_PATH,
    x_col = "image",
    y_col = "labels",
    target_size = (HEIGHT,WIDTH),
    class_mode='categorical',
    batch_size = BATCH_SIZE,
    subset = "validation",
    shuffle = True,
    seed = SEED,
    validate_filenames = False
)


test_ds = datagen.flow_from_dataframe(
    dataframe= test_df,
    directory = TEST_FOLDER,
    x_col = "image",
    y_col = 'labels',
    target_size = (HEIGHT,WIDTH),
    color_mode="rgb",
    batch_size=1,
    shuffle=False,
    seed=None,
    subset=None,    
    interpolation="box"
)


Found 4000 non-validated image filenames belonging to 12 classes.
Found 1000 non-validated image filenames belonging to 12 classes.
Found 3 validated image filenames belonging to 1 classes.


In [9]:
'''
def create_model():
    
    pretrained = efn.EfficientNetB0(include_top=False, weights="imagenet",input_shape=[HEIGHT,WIDTH, 3])
            
    x = pretrained.output
    x = tf.keras.layers.GlobalAveragePooling2D() (x)
    outputs = tf.keras.layers.Dense(NUM_CLASSES,activation="softmax", dtype='float32')(x)
        
    model = tf.keras.Model(pretrained.input, outputs)
    return model

model = create_model()
model.summary()
model.save('pp_model.h5')
'''

'\ndef create_model():\n    \n    pretrained = efn.EfficientNetB0(include_top=False, weights="imagenet",input_shape=[HEIGHT,WIDTH, 3])\n            \n    x = pretrained.output\n    x = tf.keras.layers.GlobalAveragePooling2D() (x)\n    outputs = tf.keras.layers.Dense(NUM_CLASSES,activation="softmax", dtype=\'float32\')(x)\n        \n    model = tf.keras.Model(pretrained.input, outputs)\n    return model\n\nmodel = create_model()\nmodel.summary()\nmodel.save(\'pp_model.h5\')\n'

In [10]:
from keras.models import load_model
import tensorflow_addons as tfa

model = load_model('../input/models1/pp_model.h5')

def compile_model(model, lr=0.0001):
    
    optimizer = tf.keras.optimizers.Adam(lr=lr)
    
    loss = tf.keras.losses.CategoricalCrossentropy()
    
    #tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy')
    metrics = [
    tfa.metrics.F1Score(num_classes = NUM_CLASSES,average = "macro",name = "f1_score")
    ]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model


In [11]:

METRIC = "val_f1_score"

def create_callbacks(metric = METRIC):
    
    cpk_path = './best_model.h5'
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=cpk_path,
        monitor= metric,
        mode='max',
        save_best_only=True,
        verbose=1,
    )

    reducelr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor= metric,
        mode='max',
        factor=0.1,
        patience=3,
        verbose=0
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor= metric,
        mode='max',
        patience=10, 
        verbose=1
    )
    
    callbacks = [checkpoint, reducelr, earlystop]         
    
    return callbacks


In [12]:

EPOCHS= 2
VERBOSE =1

tf.keras.backend.clear_session()

model = compile_model(model, lr=0.0001)

callbacks = create_callbacks()

history = model.fit(
                    train_ds,
                    epochs=EPOCHS,
                    validation_data = val_ds,
                    verbose=VERBOSE
                   )
model.save('best_model.h5')

Epoch 1/2
125/125 [==============================] - 1016s 8s/step - loss: 2.1701 - f1_score: 0.1338 - val_loss: 1.7556 - val_f1_score: 0.1704
Epoch 2/2
125/125 [==============================] - 918s 7s/step - loss: 1.1496 - f1_score: 0.3193 - val_loss: 1.2671 - val_f1_score: 0.2805


In [13]:
'''
from keras.models import load_model
model = load_model('best_model.h5')
'''

"\nfrom keras.models import load_model\nmodel = load_model('best_model.h5')\n"

In [14]:

answer = model.predict(test_ds)
print(answer)

sub = (np.argmax(answer, axis=1))
print(sub)

tot = []
for i in range(answer.shape[0]):
    tmp = []
    for j, c in enumerate(labels_list):
        if j == sub[i]:
            tmp.append(c)
    
    tot.append(tmp)

tot = [' '.join(t) for t in tot]

tot

[[0.12393902 0.05770484 0.04867283 0.04262815 0.07789941 0.06777168
  0.05264804 0.1257544  0.08540457 0.04269443 0.15335515 0.12152748]
 [0.11391488 0.22121306 0.06754587 0.04490794 0.07638195 0.01654615
  0.07954263 0.06785349 0.04706691 0.1373856  0.08476226 0.04287914]
 [0.07839984 0.37695923 0.01636684 0.0156265  0.04845252 0.00824423
  0.33018643 0.02666221 0.03205938 0.01981167 0.0211307  0.0261004 ]]
[10  1  1]


['scab frog_eye_leaf_spot', 'frog_eye_leaf_spot', 'frog_eye_leaf_spot']

In [15]:
test_df['labels'] = np.array(tot)

test_df

,image,labels
0,85f8cb619c66b863.jpg,scab frog_eye_leaf_spot
1,ad8770db05586b59.jpg,frog_eye_leaf_spot
2,c7b03e718489f3ca.jpg,frog_eye_leaf_spot


In [16]:
test_df.to_csv('submission.csv', index = False)